In [1]:
pip install lightgbm --install-option=--gpu

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: --install-option is deprecated because it forces pip to use the 'setup.py install' command which is itself deprecated. pip 23.1 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11358
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453


In [2]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install optuna

     ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
     ------------------------------- ------ 307.2/365.3 kB 9.6 MB/s eta 0:00:01
     -------------------------------------- 365.3/365.3 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the data
X = vectorizer.fit_transform(train_df['text'])
y = train_df['label']

# Convert the features to float64
X = csr_matrix(X, dtype='float64')

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X, y, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:15:41,666] A new study created in memory with name: no-name-f4029f90-e4ff-475a-b5f4-ee0ba38be2db
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1941034719.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

ValueError: Found input variables with inconsistent numbers of samples: [9480, 75840]

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the data
X = vectorizer.fit_transform(train_df['text'])
y = train_df['label']

# Convert the features to float64
X = X.astype('float64')

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X, y, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = X_test.astype('float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:16:47,874] A new study created in memory with name: no-name-5e4f753f-9047-4a06-b437-310a2c024777
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1782866026.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

ValueError: Found input variables with inconsistent numbers of samples: [9480, 75840]

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the data
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

# Convert the features to float64
X_train = csr_matrix(X_train, dtype='float64')

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:17:59,369] A new study created in memory with name: no-name-c644a06f-f227-488b-be4c-e082c9755d65
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1338981769.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

ValueError: Found input variables with inconsistent numbers of samples: [9480, 75840]

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the training data
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

# Convert the features to float64
X_train = csr_matrix(X_train, dtype='float64')

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the training data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:19:24,055] A new study created in memory with name: no-name-375f75d7-89d1-496d-ab60-0632bf2353ca
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\2500633042.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

ValueError: Found input variables with inconsistent numbers of samples: [9480, 75840]

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the training data
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

# Convert the features to float64
X_train = csr_matrix(X_train, dtype='float64')

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the training data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric=lambda y_true, y_pred: ('macro_f1', f1_score(y_true, y_pred, average='macro')))

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:20:21,338] A new study created in memory with name: no-name-fdf86c1d-b392-4d61-9387-b57e1f14e0be
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\2500633042.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

ValueError: Found input variables with inconsistent numbers of samples: [9480, 75840]

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the training data
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

# Convert the features to float64
X_train = csr_matrix(X_train, dtype='float64')

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclassova',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric='macro_f1')

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the training data using the best hyperparameters
params = study.best_params
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric='macro_f1')

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:21:22,276] A new study created in memory with name: no-name-dd9537fc-87b4-4d5f-b697-1ec138ed1b73
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\873452568.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fut

LightGBMError: Multiclass objective and metrics don't match

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

# Load the train data
train_df = pd.read_csv('train.csv')

# Load the test data
test_df = pd.read_csv('test.csv')

# Define the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer to the training data
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

# Convert the features to float64
X_train = csr_matrix(X_train, dtype='float64')

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 8,
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric='multi_logloss')

    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average='macro')

    return f1

# Define the Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function using Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(study.best_params)

# Train the LightGBM model on the training data using the best hyperparameters
params = study.best_params
params['objective'] = 'multiclass'
params['metric'] = 'multi_logloss'
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=10, eval_metric='multi_logloss')

# Transform the test data using the CountVectorizer
X_test = vectorizer.transform(test_df['text'])

# Convert the features to float64
X_test = csr_matrix(X_test, dtype='float64')

# Make predictions on the test data using the trained LightGBM model
y_pred = model.predict(X_test)

# Save the predictions to the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


[I 2023-03-19 03:22:51,752] A new study created in memory with name: no-name-70ffd7f3-0df2-48b5-b198-e6095ae124b6
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1026598587.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a fu

[10]	valid_0's multi_logloss: 0.626706
[20]	valid_0's multi_logloss: 0.468982
[30]	valid_0's multi_logloss: 0.406242
[40]	valid_0's multi_logloss: 0.372455
[50]	valid_0's multi_logloss: 0.351069
[60]	valid_0's multi_logloss: 0.336393


[I 2023-03-19 03:23:02,396] Trial 0 finished with value: 0.8470799542764375 and parameters: {'n_estimators': 60, 'learning_rate': 0.13529608024496062, 'num_leaves': 37, 'max_depth': 11, 'min_child_samples': 46}. Best is trial 0 with value: 0.8470799542764375.
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1026598587.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Us

[10]	valid_0's multi_logloss: 1.46805
[20]	valid_0's multi_logloss: 1.35844
[30]	valid_0's multi_logloss: 1.26932
[40]	valid_0's multi_logloss: 1.19515
[50]	valid_0's multi_logloss: 1.13113
[60]	valid_0's multi_logloss: 1.07531
[70]	valid_0's multi_logloss: 1.02595
[80]	valid_0's multi_logloss: 0.981821
[90]	valid_0's multi_logloss: 0.942492
[100]	valid_0's multi_logloss: 0.907201


[I 2023-03-19 03:23:23,760] Trial 1 finished with value: 0.6965905875254157 and parameters: {'n_estimators': 103, 'learning_rate': 0.006188938212008328, 'num_leaves': 19, 'max_depth': 18, 'min_child_samples': 13}. Best is trial 0 with value: 0.8470799542764375.
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1026598587.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\

...

[10]	valid_0's multi_logloss: 0.860642
[20]	valid_0's multi_logloss: 0.635326
[30]	valid_0's multi_logloss: 0.524539
[40]	valid_0's multi_logloss: 0.461383
[50]	valid_0's multi_logloss: 0.420064
[60]	valid_0's multi_logloss: 0.391441
[70]	valid_0's multi_logloss: 0.369858
[80]	valid_0's multi_logloss: 0.35357
[90]	valid_0's multi_logloss: 0.34104
[100]	valid_0's multi_logloss: 0.330694
[110]	valid_0's multi_logloss: 0.322214
[120]	valid_0's multi_logloss: 0.315432


[I 2023-03-19 03:52:28,048] Trial 98 finished with value: 0.8738211286418363 and parameters: {'n_estimators': 127, 'learning_rate': 0.06447971968470011, 'num_leaves': 24, 'max_depth': 17, 'min_child_samples': 36}. Best is trial 71 with value: 0.8810595533290522.
C:\Users\silvi\AppData\Local\Temp\ipykernel_8204\1026598587.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:

[10]	valid_0's multi_logloss: 0.78723
[20]	valid_0's multi_logloss: 0.57144
[30]	valid_0's multi_logloss: 0.473427
[40]	valid_0's multi_logloss: 0.418905
[50]	valid_0's multi_logloss: 0.385333
[60]	valid_0's multi_logloss: 0.361461
[70]	valid_0's multi_logloss: 0.344364
[80]	valid_0's multi_logloss: 0.331316
[90]	valid_0's multi_logloss: 0.321064
[100]	valid_0's multi_logloss: 0.313047
[110]	valid_0's multi_logloss: 0.306158
[120]	valid_0's multi_logloss: 0.300772
[130]	valid_0's multi_logloss: 0.295309
[140]	valid_0's multi_logloss: 0.291326


[I 2023-03-19 03:52:53,564] Trial 99 finished with value: 0.8679434477712842 and parameters: {'n_estimators': 142, 'learning_rate': 0.07593790342426376, 'num_leaves': 30, 'max_depth': 16, 'min_child_samples': 28}. Best is trial 71 with value: 0.8810595533290522.
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\silvi\anaconda3\envs\test\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


{'n_estimators': 149, 'learning_rate': 0.10596416395908402, 'num_leaves': 19, 'max_depth': 13, 'min_child_samples': 35}
[10]	valid_0's multi_logloss: 0.726279
[20]	valid_0's multi_logloss: 0.540229
[30]	valid_0's multi_logloss: 0.4573
[40]	valid_0's multi_logloss: 0.411331
[50]	valid_0's multi_logloss: 0.381611
[60]	valid_0's multi_logloss: 0.360758
[70]	valid_0's multi_logloss: 0.345459
[80]	valid_0's multi_logloss: 0.333706
[90]	valid_0's multi_logloss: 0.323556
[100]	valid_0's multi_logloss: 0.31559
[110]	valid_0's multi_logloss: 0.309176
[120]	valid_0's multi_logloss: 0.303208
[130]	valid_0's multi_logloss: 0.298744
[140]	valid_0's multi_logloss: 0.29493
